# AI Learning Assistant using Gemini API  
Tensorflow course – Homework 1.2.2

This notebook demonstrates an **end-to-end generative AI system** called  
**“AI Learning Assistant”**.

The system:

1. Loads small teaching texts **X** from GitHub  
   (`/1.2/data/example1–3/X.txt`).
2. Uses **Gemini API** to generate outputs **Y**:
   - summary (`y_summary`)
   - quiz (`y_test`)
   - learning plan (`y_plan`)
   - Q&A answer (`y_chat`)
3. Compares the generated outputs with the reference files in `/data`.
4. Shows how the assistant can be used on completely **new input text**.

Structure of the notebook:

- **Step 1.** Load (X, y) pairs from GitHub  
- **Step 2.** Inspect examples  
- **Step 3.** Define the Gemini-based AI Learning Assistant  
- **Step 4.** End-to-end generation for examples 1–3  
- **Step 5.** Inference on new data (interactive usage)


In [46]:
# Step 1. Load (X, y) data from GitHub

!git clone https://github.com/AdomasSim/Tensorflow-vgtu.git

import os
from pathlib import Path

BASE_PATH = "/content/Tensorflow-vgtu"

DATA_DIR = os.path.join(BASE_PATH, "1.2", "data")
PROMPTS_DIR = os.path.join(BASE_PATH, "1.2", "prompts")

print("Base path:", BASE_PATH)
print("Data directory:", DATA_DIR)
print("Data subfolders:", os.listdir(DATA_DIR))
print("Prompts directory:", PROMPTS_DIR)
print("Prompts files:", os.listdir(PROMPTS_DIR))


fatal: destination path 'Tensorflow-vgtu' already exists and is not an empty directory.
Base path: /content/Tensorflow-vgtu
Data directory: /content/Tensorflow-vgtu/1.2/data
Data subfolders: ['example3', 'example2', 'example1']
Prompts directory: /content/Tensorflow-vgtu/1.2/prompts
Prompts files: ['test_prompt_example2.txt', 'test_prompt_example3.txt', 'test_prompt_example1.txt']


In [47]:
# Utility to read UTF-8 text files

def read_text_file(path: Path) -> str:
    if not path.exists():
        raise FileNotFoundError(f"File not found: {path}")
    return path.read_text(encoding="utf-8")


def load_example(example_name: str):
    """
    Load X and all Y files for one example.
    Folder layout:
      1.2/data/exampleN/X.txt
                           y_summary.txt
                           y_test.txt
                           y_plan.txt
                           y_chat.txt
    """
    example_dir = Path(DATA_DIR) / example_name

    x_path         = example_dir / "X.txt"
    y_summary_path = example_dir / "y_summary.txt"
    y_test_path    = example_dir / "y_test.txt"
    y_plan_path    = example_dir / "y_plan.txt"
    y_chat_path    = example_dir / "y_chat.txt"

    example = {
        "name": example_name,
        "X": read_text_file(x_path),
        "y_summary": read_text_file(y_summary_path) if y_summary_path.exists() else None,
        "y_test": read_text_file(y_test_path) if y_test_path.exists() else None,
        "y_plan": read_text_file(y_plan_path) if y_plan_path.exists() else None,
        "y_chat": read_text_file(y_chat_path) if y_chat_path.exists() else None,
    }
    return example


# Load all three examples

example_names = ["example1", "example2", "example3"]
examples = [load_example(name) for name in example_names]

for ex in examples:
    print(f"{ex['name']}: |X| = {len(ex['X'])} characters")


example1: |X| = 313 characters
example2: |X| = 317 characters
example3: |X| = 276 characters


## Step 2. Inspect one training example

We briefly inspect `example1` to understand what kind of data the agent will use:

- `X.txt` – short teaching text.
- `y_summary.txt` – target summary.
- `y_test.txt` – target quiz (questions + options + correct answer).
- `y_plan.txt` – target learning plan.
- `y_chat.txt` – example of question & answer (Q&A).


In [48]:
ex = examples[0]  # example1

print("=== X.txt (teaching text) ===")
print(ex["X"])
print("\n=== y_summary.txt (reference) ===")
print(ex["y_summary"])
print("\n=== y_test.txt (reference) – first 400 chars ===")
print(ex["y_test"][:400], "...")
print("\n=== y_plan.txt (reference) ===")
print(ex["y_plan"])
print("\n=== y_chat.txt (reference) ===")
print(ex["y_chat"])


=== X.txt (teaching text) ===
Dvejetainė skaičiavimo sistema naudoja tik du skaitmenis: 0 ir 1.
Kiekviena pozicija reiškia dvejeto laipsnį: 1, 2, 4, 8, 16 ir t.t.
Pavyzdžiui, skaičius 5 dvejetainėje sistemoje yra 101, nes 4 + 1 = 5.
Kompiuteriai naudoja dvejetainę sistemą, nes elektroniniai komponentai turi dvi būsenas: įjungta ir išjungta.


=== y_summary.txt (reference) ===
Santrauka:
Dvejetainė sistema naudoja tik skaitmenis 0 ir 1, o kiekviena pozicija atitinka dvejeto laipsnį.
Skaičius 5 dvejetainėje sistemoje užrašomas 101, nes 4 + 1 = 5.
Kompiuteriai remiasi dvejetaine sistema, nes elektroniniai komponentai turi dvi aiškias būsenas - įjungta ir išjungta.


=== y_test.txt (reference) – first 400 chars ===
Testas:

1) Kiek skaitmenų naudoja dvejetainė skaičiavimo sistema?
   A) 1
   B) 2
   C) 10
   D) 16
   Teisingas atsakymas: B
   Paaiškinimas: Dvejetainė sistema naudoja tik 0 ir 1, todėl yra dvinarė.

2) Kokia yra skaičiaus 5 dvejetainė išraiška?
   A) 100
   B) 101
   C) 110


## Step 3. Define the Gemini-based AI Learning Assistant

We will use **Gemini API** as a generative model.  
For each input text **X**, the assistant will:

1. Generate a short **summary**.
2. Generate a **quiz** with several questions and answer options.
3. Generate a simple **learning plan**.
4. Answer a **Q&A question** about the text.

Below we configure Gemini and define prompt templates.


In [49]:
!pip install -q google-generativeai

import google.generativeai as genai
from google.colab import userdata
# IMPORTANT:
# Set GEMINI_API_KEY in the Colab environment before running:
#   import os
#   os.environ["GEMINI_API_KEY"] = "YOUR_API_KEY_HERE"
# (do NOT commit the key to GitHub)

api_key = userdata.get("GEMINI_API_KEY")
if not api_key:
    raise ValueError("GEMINI_API_KEY is not set in Colab secrets. Patikrink pavadinimą 'GEMINI_API_KEY'.")

genai.configure(api_key=api_key)

model = genai.GenerativeModel("gemini-2.5-flash")
print("Gemini model is ready.")
print("google-generativeai version:", genai.__version__)


Gemini model is ready.
google-generativeai version: 0.8.5


In [50]:
for m in genai.list_models():
    print(m.name)

models/embedding-gecko-001
models/gemini-2.5-flash
models/gemini-2.5-pro
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
models/gemini-2.5-flash-preview-tts
models/gemini-2.5-pro-preview-tts
models/learnlm-2.0-flash-experimental
models/gemma-3-1b-it
models/gemma-3-4b-it
models/gemma-3-12b-it
models/gemma-3-27b-it
models/gemma-3n-e4b-it
models/gemma-3n-e2b-it
models/gemini-flash-latest
models/gemini-flash-lite-latest
models/gemini-pro-latest
models/gemini-2.5-flash-lite
models/gemini-2.5-flash-image-preview
models/gemini-2.5-flash-image
models/gemini-2.5-flash-preview-09-2025
models/gemini-2.5-flash-lite-preview-09-2025
models/gemini-3-pro-preview
models/gemini-3-pro-image-preview
models/nano-banana-pro-preview
models/gemini-robotics-er

In [51]:
BASE_SYSTEM_PROMPT = """
Tu esi AI mokymosi asistentas.
Tavo tikslas – aiškiai ir trumpai paaiškinti pateiktą mokymo medžiagą.
Laikykis nurodyto formato ir nerašyk nereikalingų įžangų.
"""


def build_summary_prompt(document_text: str) -> str:
    return f"""{BASE_SYSTEM_PROMPT}

Užduotis: remdamasis pateiktu tekstu, parašyk trumpą santrauką.

Reikalavimai:
- 3–6 sakiniai
- tik svarbiausios idėjos
- nekurk naujų faktų

Mokymo medžiaga:
\"\"\"{document_text}\"\"\""""


def build_test_prompt(document_text: str) -> str:
    return f"""{BASE_SYSTEM_PROMPT}

Užduotis: remdamasis pateiktu tekstu, sukurk testą.

Reikalavimai:
- 3–5 klausimai
- kiekvienam klausimui 4 atsakymo variantai (A, B, C, D)
- aiškiai parašyk teisingą atsakymą
- pridėk trumpą paaiškinimą

Mokymo medžiaga:
\"\"\"{document_text}\"\"\""""


def build_plan_prompt(document_text: str) -> str:
    return f"""{BASE_SYSTEM_PROMPT}

Užduotis: sudaryk mokymosi planą šiai medžiagai išmokti.

Reikalavimai:
- 5 numeruoti žingsniai
- kiekviename žingsnyje:
  - ką daryti,
  - kiek laiko maždaug skirti,
  - koks žingsnio tikslas

Mokymo medžiaga:
\"\"\"{document_text}\"\"\""""


def build_chat_system_prompt(document_text: str) -> str:
    return f"""{BASE_SYSTEM_PROMPT}

Toliau studentas užduos klausimus apie šią mokymo medžiagą.
Atsakyk tik remdamasis žemiau pateiktu tekstu.

Mokymo medžiaga:
\"\"\"{document_text}\"\"\""""


In [55]:
def generate_summary(document_text: str) -> str:
    prompt = build_summary_prompt(document_text)
    response = model.generate_content(prompt)
    return response.text.strip()


def generate_test(document_text: str) -> str:
    prompt = build_test_prompt(document_text)
    response = model.generate_content(prompt)
    return response.text.strip()


def generate_plan(document_text: str) -> str:
    prompt = build_plan_prompt(document_text)
    response = model.generate_content(prompt)
    return response.text.strip()


def chat_about_document(text, question):
    prompt = f"""
Tu esi naudingas mokytojas.

Mokomoji medžiaga:
{text}

Klausimas apie šią medžiagą:
{question}

Atsakyk aiškiai, trumpai ir lietuviškai.
"""
    response = model.generate_content(prompt)
    return response.text



In [53]:
def parse_chat_file(y_chat_text: str):
    """
    Extract (question, answer) from y_chat.txt formatted roughly as:

    Klausimas:
    ...

    AI atsakymas:
    ...
    """
    lines = [ln.strip() for ln in y_chat_text.splitlines() if ln.strip()]
    question_lines = []
    answer_lines = []
    mode = None

    for ln in lines:
        lower = ln.lower()
        if lower.startswith("klausimas"):
            mode = "q"
            continue
        if lower.startswith("ai atsakymas"):
            mode = "a"
            continue

        if mode == "q":
            question_lines.append(ln)
        elif mode == "a":
            answer_lines.append(ln)

    question = " ".join(question_lines)
    answer = " ".join(answer_lines)
    return question, answer


# Quick sanity check for example1
q1, a1 = parse_chat_file(examples[0]["y_chat"])
print("Question from y_chat.txt:")
print(q1)
print("\nAnswer from y_chat.txt:")
print(a1)


Question from y_chat.txt:
Kodėl dvejetainėje sistemoje užtenka tik dviejų skaitmenų?

Answer from y_chat.txt:
Dvejetainėje sistemoje užtenka dviejų skaitmenų – 0 ir 1 – nes ji paremta dvejeto laipsniais. Kiekviena pozicija atspindi, ar atitinkamas dvejeto laipsnis yra įtrauktas (1) ar ne (0) į skaičiaus sumą. Tai yra patogu kompiuteriams, nes elektroniniai komponentai lengvai realizuoja dvi būsenas: yra srovė (1) arba jos nėra (0).


## Step 4. End-to-End generation for examples 1–3

For each example we:

1. Read **X.txt** (teaching text).
2. Use Gemini to generate:
   - summary,
   - quiz,
   - learning plan,
   - Q&A answer.
3. Print the **generated** result and the **reference** result from `/data`
   side-by-side for qualitative comparison.

This demonstrates a complete end-to-end pipeline from data to AI output.


In [58]:
for ex in examples:
    print("=" * 100)
    print(f"EXAMPLE: {ex['name']}")
    print("=" * 100)

    print("\n--- X (teaching text) – first 400 characters ---")
    print(ex["X"][:400])
    print()

    # 1) Summary
    print(">>> Gemini summary:")
    y_summary_pred = generate_summary(ex["X"])
    print(y_summary_pred, "\n")

    if ex["y_summary"] is not None:
        print(">>> Reference y_summary.txt:")
        print(ex["y_summary"], "\n")

    # 2) Test / quiz
    print(">>> Gemini quiz (test):")
    y_test_pred = generate_test(ex["X"])
    print(y_test_pred, "\n")

    if ex["y_test"] is not None:
        print(">>> Reference y_test.txt:")
        print(ex["y_test"], "\n")

    # 3) Learning plan
    print(">>> Gemini learning plan:")
    y_plan_pred = generate_plan(ex["X"])
    print(y_plan_pred, "\n")

    if ex["y_plan"] is not None:
        print(">>> Reference y_plan.txt:")
        print(ex["y_plan"], "\n")

    # 4) Q&A chat
    if ex["y_chat"] is not None:
        question, target_answer = parse_chat_file(ex["y_chat"])
    else:
        question = "Paaiškink pagrindinę šios medžiagos idėją paprastais žodžiais."
        target_answer = None

    print(">>> Q&A demo:")
    print("Question:", question)
    y_chat_pred = chat_about_document(ex["X"], question)
    print("Gemini answer:")
    print(y_chat_pred, "\n")

    if target_answer is not None:
        print(">>> Reference answer from y_chat.txt:")
        print(target_answer, "\n")


EXAMPLE: example1

--- X (teaching text) – first 400 characters ---
Dvejetainė skaičiavimo sistema naudoja tik du skaitmenis: 0 ir 1.
Kiekviena pozicija reiškia dvejeto laipsnį: 1, 2, 4, 8, 16 ir t.t.
Pavyzdžiui, skaičius 5 dvejetainėje sistemoje yra 101, nes 4 + 1 = 5.
Kompiuteriai naudoja dvejetainę sistemą, nes elektroniniai komponentai turi dvi būsenas: įjungta ir išjungta.


>>> Gemini summary:
Dvejetainė skaičiavimo sistema naudoja tik du skaitmenis – 0 ir 1. Kiekviena pozicija šioje sistemoje reiškia dvejeto laipsnį, pavyzdžiui, 1, 2, 4 ir t.t. Skaičius 5 dvejetainėje sistemoje yra 101, nes 4 + 1 = 5. Kompiuteriai naudoja dvejetainę sistemą dėl to, kad jų elektroniniai komponentai gali turėti tik dvi būsenas: įjungta arba išjungta. 

>>> Reference y_summary.txt:
Santrauka:
Dvejetainė sistema naudoja tik skaitmenis 0 ir 1, o kiekviena pozicija atitinka dvejeto laipsnį.
Skaičius 5 dvejetainėje sistemoje užrašomas 101, nes 4 + 1 = 5.
Kompiuteriai remiasi dvejetaine sistema, nes ele

## Step 5. Inference on new data (interactive usage)

Finally, we show how the **AI Learning Assistant** can be used on completely
new text that is not stored in `/data`.

We create a small teaching paragraph `new_X` and ask the assistant to:

- create a summary,
- suggest a quiz,
- propose a learning plan,
- answer a custom question from the user.


In [59]:
# Define a completely new teaching text (not in GitHub data)
new_X = """
Dirbtinis intelektas (DI) – tai kompiuterių mokslo sritis, kuri siekia sukurti sistemas, galinčias atlikti užduotis, reikalaujančias žmogaus intelekto: mokymąsi, sprendimų priėmimą, kalbos supratimą ar vaizdų atpažinimą. DI pagrindas yra duomenys – kuo daugiau kokybiškos informacijos turi sistema, tuo tiksliau ji gali prognozuoti, klasifikuoti ar generuoti naują turinį.

DI galima suskirstyti į kelias kategorijas. Silpnas DI (angl. narrow AI) atlieka tik vieną konkrečią funkciją, pvz., rekomenduoja filmus, atpažįsta veidus ar prognozuoja kainas. Stiprus DI (angl. general AI) – teorinė koncepcija, kurioje sistema gebėtų mąstyti kaip žmogus, tačiau tokios technologijos šiandien dar nėra sukurtos.

Viena svarbiausių DI šakų – mašininis mokymasis. Tai metodas, kai modeliai mokomi iš duomenų, ieškodami pasikartojančių dėsningumų. Dar gilesnė sritis – gilusis mokymasis, paremtas dirbtiniais neuroniniais tinklais, kurie sudaryti iš daug sluoksnių ir imituoja žmogaus smegenų veikimą. Tokie modeliai geba analizuoti sudėtingus signalus, pvz., kalbą, tekstą, vaizdus ar garsą.

Šiuolaikinės generatyvinės DI sistemos, tokios kaip „ChatGPT“ ar „Gemini“, ne tik analizuoja informaciją, bet ir sukuria naują turinį: rašo tekstus, kuria vaizdus, programuoja ar net generuoja mokymosi planus. Nors jos labai naudingos, svarbu suprasti, kad jos gali klysti, todėl žmogaus priežiūra ir kritinis mąstymas išlieka būtini.
"""

print("=== New X text ===")
print(new_X)

print("\n--- Summary ---")
print(generate_summary(new_X))

print("\n--- Quiz ---")
print(generate_test(new_X))

print("\n--- Learning plan ---")
print(generate_plan(new_X))

print("\n--- Q&A ---")
user_question = "Kuo skiriasi GET ir POST užklausos šioje medžiagoje?"
print("User question:", user_question)
print("Assistant answer:")
print(chat_about_document(new_X, user_question))


=== New X text ===

Dirbtinis intelektas (DI) – tai kompiuterių mokslo sritis, kuri siekia sukurti sistemas, galinčias atlikti užduotis, reikalaujančias žmogaus intelekto: mokymąsi, sprendimų priėmimą, kalbos supratimą ar vaizdų atpažinimą. DI pagrindas yra duomenys – kuo daugiau kokybiškos informacijos turi sistema, tuo tiksliau ji gali prognozuoti, klasifikuoti ar generuoti naują turinį.

DI galima suskirstyti į kelias kategorijas. Silpnas DI (angl. narrow AI) atlieka tik vieną konkrečią funkciją, pvz., rekomenduoja filmus, atpažįsta veidus ar prognozuoja kainas. Stiprus DI (angl. general AI) – teorinė koncepcija, kurioje sistema gebėtų mąstyti kaip žmogus, tačiau tokios technologijos šiandien dar nėra sukurtos.

Viena svarbiausių DI šakų – mašininis mokymasis. Tai metodas, kai modeliai mokomi iš duomenų, ieškodami pasikartojančių dėsningumų. Dar gilesnė sritis – gilusis mokymasis, paremtas dirbtiniais neuroniniais tinklais, kurie sudaryti iš daug sluoksnių ir imituoja žmogaus smegen

## Conclusions

In this notebook we implemented an **end-to-end generative AI system**:

- Input **X**: short teaching texts stored in GitHub (`/1.2/data/example1–3/X.txt`).
- Output **Y**: summary, quiz, learning plan, and Q&A answer generated using **Gemini API**.
- We compared Gemini outputs with reference files (`y_summary.txt`, `y_test.txt`, `y_plan.txt`, `y_chat.txt`).
- We also showed how the same agent can be used on completely new data
  outside the training examples.

This Colab notebook can now be saved as `.ipynb` and uploaded to the
`Tensorflow-vgtu` GitHub repository as the deliverable for task **1.2.2**.
